In [1]:
#----------------------fitbit data----------
import pandas as pd
from imp import source_from_cache
from google.cloud import bigquery

#Read files
hourly_steps_file = "./data/hourlySteps_merged.csv"
hourly_steps = pd.read_csv(hourly_steps_file, header=0)

hourly_calories_file = './data/hourlyCalories_merged.csv'
hourly_calories = pd.read_csv(hourly_calories_file, header=0)

#merge data sets
steps_cal_df = pd.merge(hourly_steps, hourly_calories, on=['ActivityHour', 'Id'], how='left')
steps_cal_df['ActivityHour']=pd.to_datetime(steps_cal_df["ActivityHour"], format="%m/%d/%Y %I:%M:%S %p", utc=True)

#convert acivity hour to readable datetime object by BigQuery and drop duplicates
steps_cal_df['ActivityHour']=pd.to_datetime(steps_cal_df["ActivityHour"], format="%m/%d/%Y %I:%M:%S %p", utc=True)
steps_cal_df.drop_duplicates(ignore_index=True)

#write to csv file
steps_cal_df.to_csv("./data/hourly_calories", index=False)

# Construct a BigQuery client object to upload csv to google cloud console
client = bigquery.Client()
job_config = bigquery.LoadJobConfig(
    source_format="CSV",
    autodetect=True,
    create_disposition="CREATE_IF_NEEDED",
    write_disposition="WRITE_TRUNCATE",
)
table_id = "deb-01-372116.fit_bit.hourly_calories"
file_name = "./data/hourly_calories"
with open(file_name, "rb") as fp:
    job = client.load_table_from_file(fp, table_id, job_config=job_config)

job.result()

table = client.get_table(table_id)
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

/home/reedandrew18/BigQuery-and-the-Cloud/venv-codereview/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  This is separate from the ipykernel package so we can avoid doing imports until


Loaded 22099 rows and 4 columns to deb-01-372116.fit_bit.hourly_calories
